In [1]:
import sys
import os
print(sys.path)

In [2]:
!pwd

In [6]:
import torch
import matplotlib.pyplot as plt
from torch import optim, distributions, nn
from tqdm import tqdm
from gpzoo.kernels import RBF
from gpzoo.gp import VNNGP, SVGP
from gpzoo.likelihoods import GaussianLikelihood

In [7]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

In [8]:
N = 10000
X = distributions.Uniform(-5, 5).sample((N, ))
y = 2*torch.sin(2*X) + distributions.Normal(0.0, 0.1).sample((N,))

In [9]:
plt.scatter(X, y)

In [6]:
M = 1000
kernel = RBF(sigma=1.0, lengthscale=5.0)
gp = VNNGP(kernel, dim=1, M=M, jitter=1e-3)
gp.Z = nn.Parameter(10.0*torch.rand((M, 1))-5.0) #choose inducing points
model = GaussianLikelihood(gp)

In [7]:
model.to(device)

In [8]:
X_train = (X[:, None]).to(device)
y_train = y.to(device)

In [9]:
def fit(model, X, y, optimizer, epochs=1000, E=20):
    losses = []
    for it in tqdm(range(epochs)):
        optimizer.zero_grad()
        pY, qF, qU, pU = model.forward(X, E=E)
        ELBO = (pY.log_prob(y)).mean(axis=0).sum()
        ELBO -= torch.sum(distributions.kl_divergence(qU, pU))
        loss = -ELBO
        loss.backward()
        optimizer.step()
        losses.append(loss.item())

    print("finished Training")
    return losses

In [84]:
model.gp.K = 5

In [11]:
optimizer = optim.Adam(params= model.parameters(), lr=1e-2)

In [12]:
losses = fit(model, X_train, y_train, optimizer, epochs=2000, E=200)

In [13]:
plt.plot(losses)

In [54]:
X_test = torch.linspace(-6, 6, 150)

In [86]:
qF, _, _ = model.gp(X_test[:, None].to(device), verbose=True)
mean_test = qF.mean.detach().cpu()

In [87]:
plt.plot(X_test, mean_test, color='black')
plt.scatter(X, y, alpha=0.5)

In [94]:
samples = qF.sample((3,))
plt.plot(X_test, samples.detach().cpu().T, alpha=0.5);
plt.plot(X_test, mean_test, color='black');

In [95]:
model.gp.kernel.lengthscale

In [96]:
model.gp.kernel.sigma

In [97]:
model.noise